# **Chapter 4. Examples and Few-Shot Prompting** 

There are a number of well-known prompting techniques that use examples, such as zero-shot, one-shot, and few-shot prompting. By understanding each approach and the principles of designing effective examples, you can significantly enhance the performance of Solar. 

This chapter is based on Prompt `Type C`:  Instruction + Context + **Example** + Output 

- [Zero-Shot Prompting](#zeroshot)
- [One-Shot Prompting](#oneshot)
- [Few-Shot Prompting](#fewshot)

**Set up**

In [12]:
from openai import OpenAI

# Retrieve the UPSTAGE_API_KEY variable from the IPython store
%store -r UPSTAGE_API_KEY

client = OpenAI(
    api_key= UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1/solar"
)

config_model = {
    "model": "solar-pro",
    "max_tokens": 2000,
    "temperature": 0.7,
    "top_p": 0.9,
}

def get_completion(messages, system_prompt="", config=config_model):
    try:
        if system_prompt:
            messages = [{"role": "system", "content": system_prompt}] + messages

        message = client.chat.completions.create(messages=messages, **config)
        return message.choices[0].message.content
    
    except Exception as e:
        print(f"Error during API call: {e}")
        return None

---

<a id="zeroshot"></a>
## **4.1 Zero-Shot Prompting**

### **4.1.1 Introduction**

A technique where the model is asked to perform a task without any prior examples. There are tons of ways to craft zero-shot prompting. This is one of the powerful features of language models. It allows for performing various tasks with minimal information.

### **4.1.2 Example**

In [17]:
# Prompt:
message = [
    {
        "role":"user",
        "content": "Classify the following text as positive, negative, or neutral. Text: I thought the macaron flavor was just okay. Sentiment: { }"
    }
]

response = get_completion(messages=message)
print(response)


Sentiment: Neutral


### **4.1.3 Practice**

→ Translate the term 'artificial tears' from English into Japanese. 

In [ ]:
# Prompt:
message = [
    {
        "role":"user",
        "content": " " # ←- Insert your prompt here.
    }
]

response = get_completion(messages=message)
print(response)


---

<a id="oneshot"></a>
## **4.2 One-Shot Prompting**

### **4.2.1 Introduction**

One-shot prompting is a technique where the model is provided with only one example before it’s asked to complete the task.

### **4.2.2 Example**

In [18]:
# Prompt:
message = [
    {
        "role":"user",
        "content": "Translate the following sentence into French: I like reading books."
    },
    {
        "role":"assistant",
        "content": "J'aime lire des livres."
    },
    {
        "role":"user",
        "content": "Translate the following sentence into French: I love studying language"
    },
]

response = get_completion(messages=message)
print(response)

J'adore étudier les langues.


### **4.2.3 Practice**

→ Translate the term 'artificial tears' from English to Japanese using one-shot prompting.

In [ ]:
# Prompt:
message = [
    {" "} # ←- Insert your prompt here.
]

response = get_completion(messages=message)
print(response)

---

<a id="fewshot"></a>
## **4.3 Few-Shot Prompting**

### **4.3.1 Introduction**

Few-shot prompting is a technique where the model is given a handful of examples—typically two to five—before it is asked to complete the task. This approach offers the model multiple references for the type of output desired, providing more context and making the output more accurate or in line with specific patterns.

> It's important to note that **few-shot learning** and **few-shot prompting** are distinct concepts. Few-shot learning is a broader machine learning approach focused on adapting model parameters with only a few examples. In contrast, **few-shot prompting** specifically applies to prompt design in generative AI, where model parameters remain unchanged.

### **4.3.2 Examples**

In [19]:
# Prompt:
message = [
    {
        "role": "user",
        "content": "2+10:"
    },
    {
        "role": "assistant",
        "content": "twelve"
    },
    {
        "role": "user",
        "content": "4+52:"
    },
    {
        "role": "assistant",
        "content": " fifty-six"
    },
    {
        "role": "user",
        "content": "100+301:"
    }
]

response = get_completion(messages=message)
print(response)

 four-hundred-and-one


We would like to highlight three ways to design effective examples when using Solar. 

- [**Exemplar Quantity**](#quant)
- [**Exemplar Similarity**](#sim)
- [**Exemplar Format**](#format)

<a id="quant"></a>
#### **(1) Exemplar Quantity**

Mostly, the quantity of exemplars in the prompt generally improves model
performance. 

In [20]:
# Prompt:
message = [
    {
        "role": "user",
        "content": "The cat sleeps."
    },
    {
        "role": "assistant",
        "content": """Sentence: "The cat sleeps."
Subject: "The cat"
Verb: "sleeps"
Object: None
Analysis: Simple subject-verb sentence structure with a single actor performing an action."""
    },
    {
        "role": "user",
        "content": "She reads books."
    },
    {
        "role": "assistant",
        "content": """Sentence: "She reads books."
Subject: "She"
Verb: "reads"
Object: "books"
Analysis: Subject-verb-object structure, indicating the action is directed toward an object."""
    },
    {
        "role": "user",
        "content": "They dance gracefully."
    },
    {
        "role": "assistant",
        "content": """Sentence: "They dance gracefully."
Subject: "They"
Verb: "dance"
Adverb: "gracefully"
Analysis: Subject-verb-adverb structure, with the adverb modifying how the action is performed."""
    },
    {
        "role": "user",
        "content": "The sun rises in the east."
    },
    {
        "role": "assistant",
        "content": """Sentence: "The sun rises in the east."
Subject: "The sun"
Verb: "rises"
Prepositional Phrase: "in the east"
Analysis: Subject-verb-prepositional phrase structure, adding context about the action's location."""
    },
    {
        "role": "user",
        "content": "Thick smog chokes northern India and eastern Pakistan ahead of Diwali."
    }
]

response = get_completion(messages=message)
print(response)

Sentence: "Thick smog chokes northern India and eastern Pakistan ahead of Diwali."
Subject: "Thick smog"
Verb: "chokes"
Object: "northern India and eastern Pakistan"
Prepositional Phrase: "ahead of Diwali"
Analysis: Subject-verb-object-prepositional phrase structure, providing a detailed description of the action and its context.


In [22]:
# without an example

# Prompt:
message = [
    {
        "role":"user",
        "content": """Please analyze and define the sentence components in the following sentence.
        sentence: "Thick smog chokes northern India and eastern Pakistan ahead of Diwali."
        """
    }
]

response = get_completion(messages=message)
print(response)

In the sentence "Thick smog chokes northern India and eastern Pakistan ahead of Diwali," we have:

1. "Thick smog" - subject (noun phrase)
2. "chokes" - predicate (verb)
3. "northern India and eastern Pakistan" - direct object (noun phrase)
4. "ahead of Diwali" - adverbial phrase (time)


> Observation: If you provide an example, you can see the model respond in the format used in the example. Even with complex and varied sentence structures, using many examples to guide the model helps it perform the task well. 

<a id="sim"></a>
#### **(2) Exemplar Similarity**

Select exemplars that are similar to your task. For example, if you are summarizing a news article, using exemplars in the same format will yield the best results you desire.

In the following prompt example, you can see that the summary results use **a consistent response format** as an example. By increasing the consistency of the format, you can achieve more accurate results.

In [27]:
# When examples have the similarities
# Prompt:
message = [
    {
        "role":"user",
        "content": """Read the following text and summarize the key points. Once you finish the summary, organize it as follows:

#Keyword 1 / Related key point in one short phrase./
#Keyword 2 / Related key point in one short phrase./
#Keyword 3 / Related key point in one short phrase./

<text>
To be considered open source under the OSAID, an AI model has to provide enough information about its design so that a person could “substantially” recreate it. The model must also disclose any pertinent details about its training data, including the provenance, how the data was processed, and how it can be obtained or licensed.
“An open source AI is an AI model that allows you to fully understand how it’s been built,” Maffulli said. “That means that you have access to all the components, such as the complete code used for training and data filtering.”
The OSAID also lays out usage rights developers should expect with open source AI, like the freedom to use the model for any purpose and modify it without having to ask anyone’s permission. “Most importantly, you should be able to build on top,” added Maffulli.
The OSI has no enforcement mechanisms to speak of. It can’t pressure developers to abide by or follow the OSAID. But it does intend to flag models described as “open source” but which fall short of the definition.
“Our hope is that when someone tries to abuse the term, the AI community will say, ‘We don’t recognize this as open source,’ and it gets corrected,” Maffulli said. Historically, this has had mixed results, but it isn’t entirely without effect.
Many startups and big tech companies, most prominently Meta, have employed the term “open source” to describe their AI model release strategies — but few meet the OSAID’s criteria. For example, Meta mandates that platforms with over 700 million monthly active users request a special license to use its [Llama](https://techcrunch.com/2024/09/08/meta-llama-everything-you-need-to-know-about-the-open-generative-ai-model/) models.
Maffulli has been [openly critical](https://www.ft.com/content/397c50d8-8796-4042-a814-0ac2c068361f) of Meta’s decision to call its models “open source.” After discussions with the OSI, Google and Microsoft agreed to drop their use of the term for models that aren’t fully open, but Meta hasn’t, he said.
Stability AI, which has long advertised its models as “open,” requires that businesses making more than $1 million in revenue obtain an enterprise license. And French AI upstart Mistral’s license bars the use of certain models and outputs for commercial ventures.
A [study](https://www.wired.com/story/the-myth-of-open-source-ai/) last August by researchers at the Signal Foundation, the nonprofit AI Now Institute, and Carnegie Mellon found that many “open source” models are basically open source in name only. The data required to train the models is kept secret, the compute power needed to run them is beyond the reach of many developers, and the techniques to fine-tune them are intimidatingly complex.
Instead of democratizing AI, these “open source” projects tend to entrench and expand centralized power, the study’s authors concluded. Indeed, Meta’s Lllama models have [racked up](https://venturebeat.com/ai/meta-leads-open-source-ai-boom-llama-downloads-surge-10x-year-over-year/) hundreds of millions of downloads, and Stability [claims](https://www.prnewswire.com/news-releases/stability-ai-secures-significant-new-investment-from-world-class-investor-group-and-appoints-prem-akkaraju-as-ceo-302181923.html) that its models power up to 80% of all AI-generated imagery.
</text>"""
    }
]

response = get_completion(messages=message)
print(response)

#Keyword 1 / Open source AI definition / An AI model that provides enough information about its design, training data, and usage rights to allow substantial recreation and modification without permission./
#Keyword 2 / OSAID / The Open Source AI Definition laid out by the OSI, which includes criteria for open source AI and usage rights./
#Keyword 3 / AI community enforcement / The OSI relies on the AI community to enforce the OSAID definition, flagging models that fall short of the criteria./


In [32]:
# When examples don't have the similarities
# Prompt:
message = [
    {
        "role":"user",
        "content": """Read the following text and summarize the key points. Once you finish the summary, organize it as follows:

Related key point in one short phrase./Keyword 1
Keyword 2 Related key point in two short phrases.
Keyword 3/ Related key point in three short phrases.

<text>
To be considered open source under the OSAID, an AI model has to provide enough information about its design so that a person could “substantially” recreate it. The model must also disclose any pertinent details about its training data, including the provenance, how the data was processed, and how it can be obtained or licensed.
“An open source AI is an AI model that allows you to fully understand how it’s been built,” Maffulli said. “That means that you have access to all the components, such as the complete code used for training and data filtering.”
The OSAID also lays out usage rights developers should expect with open source AI, like the freedom to use the model for any purpose and modify it without having to ask anyone’s permission. “Most importantly, you should be able to build on top,” added Maffulli.
The OSI has no enforcement mechanisms to speak of. It can’t pressure developers to abide by or follow the OSAID. But it does intend to flag models described as “open source” but which fall short of the definition.
“Our hope is that when someone tries to abuse the term, the AI community will say, ‘We don’t recognize this as open source,’ and it gets corrected,” Maffulli said. Historically, this has had mixed results, but it isn’t entirely without effect.
Many startups and big tech companies, most prominently Meta, have employed the term “open source” to describe their AI model release strategies — but few meet the OSAID’s criteria. For example, Meta mandates that platforms with over 700 million monthly active users request a special license to use its [Llama](https://techcrunch.com/2024/09/08/meta-llama-everything-you-need-to-know-about-the-open-generative-ai-model/) models.
Maffulli has been [openly critical](https://www.ft.com/content/397c50d8-8796-4042-a814-0ac2c068361f) of Meta’s decision to call its models “open source.” After discussions with the OSI, Google and Microsoft agreed to drop their use of the term for models that aren’t fully open, but Meta hasn’t, he said.
Stability AI, which has long advertised its models as “open,” requires that businesses making more than $1 million in revenue obtain an enterprise license. And French AI upstart Mistral’s license bars the use of certain models and outputs for commercial ventures.
A [study](https://www.wired.com/story/the-myth-of-open-source-ai/) last August by researchers at the Signal Foundation, the nonprofit AI Now Institute, and Carnegie Mellon found that many “open source” models are basically open source in name only. The data required to train the models is kept secret, the compute power needed to run them is beyond the reach of many developers, and the techniques to fine-tune them are intimidatingly complex.
Instead of democratizing AI, these “open source” projects tend to entrench and expand centralized power, the study’s authors concluded. Indeed, Meta’s Lllama models have [racked up](https://venturebeat.com/ai/meta-leads-open-source-ai-boom-llama-downloads-surge-10x-year-over-year/) hundreds of millions of downloads, and Stability [claims](https://www.prnewswire.com/news-releases/stability-ai-secures-significant-new-investment-from-world-class-investor-group-and-appoints-prem-akkaraju-as-ceo-302181923.html) that its models power up to 80% of all AI-generated imagery.

</text>"""
    }
]

responses = []
for i in range(3):
    response = get_completion(messages=message)
    responses.append(response)

- As seen in the following results, prompts with lower similarity among examples yield a wider variety of outcomes.

In [33]:
for i in range(3):
    print(f"n = {i+1}")
    print(responses[i])
    print("\n")

n = 1
Key point: Open Source AI Definition and Criteria
- OSAID: AI model design, training data details, usage rights

OSAID, OSI, and Maffulli: Open Source AI Enforcement and Criticism
- OSI flags non-compliant models, mixed results
- Maffulli critical of Meta, Google, Microsoft, Stability AI, Mistral

AI Community and Centralized Power
- Study: "Open source" models often not truly open, entrench centralized power
- Meta, Stability AI dominate AI model downloads


n = 2
Key point: Open Source AI Model Definition and Challenges

Open Source AI Model Definition: A model that allows for substantial recreation, discloses training data details, and grants usage rights.

Challenges in Open Source AI: Misuse of term, limited accessibility, and entrenchment of centralized power.

1. Open Source AI Model Definition: An AI model that enables substantial recreation, reveals training data specifics, and provides usage rights.
2. Challenges in Open Source AI:
   - Misuse of term: Companies like Me

<a id="format"></a>
#### **(3) Exemplar Format**

Use a common template. Optimal format may vary across tasks. 

There is evidence indicating that formats frequently found in the training data tend to result in improved performance.

In [34]:
# Prompt:
message = [
    {
        "role":"user",
        "content": """Identify the entities referenced in the text below. 

Please extract the following four types of entities:
- company names: 
- personal names: 
- specific topics:
- themes:

<text>
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, demonstrating superior performance in various natural language processing (NLP) tasks. Inspired by recent efforts to efficiently up-scale LLMs, we present a method for scaling LLMs called depth up-scaling (DUS), which encompasses depthwise scaling and continued pretraining. In contrast to other LLM up-scaling methods that use mixture-of-experts, DUS does not require complex changes to train and inference efficiently. We show experimentally that DUS is simple yet effective in scaling up high-performance LLMs from small ones. Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, a variant fine-tuned for instruction-following capabilities, surpassing Mixtral-8x7B-Instruct. SOLAR 10.7B is publicly available under the Apache 2.0 license, promoting broad access and application in the LLM field.
</text>"""
    }
]

response = get_completion(messages=message)
print(response)

company names:
- SOLAR

personal names:
- (none)

specific topics:
- depth up-scaling (DUS)
- mixture-of-experts
- SOLAR 10.7B-Instruct
- Mixtral-8x7B-Instruct
- Apache 2.0 license

themes:
- large language models (LLMs)
- natural language processing (NLP) tasks
- scaling LLMs
- instruction-following capabilities


In [37]:
# When not using a standard format
# Prompt:
message = [
    {
        "role":"user",
        "content": """Identify the entities referenced in the text below. 

Please extract the following four types of entities:
% % company names
% % personal names 
% % specific topics% % 
% %  themes % % 

~text~
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, demonstrating superior performance in various natural language processing (NLP) tasks. Inspired by recent efforts to efficiently up-scale LLMs, we present a method for scaling LLMs called depth up-scaling (DUS), which encompasses depthwise scaling and continued pretraining. In contrast to other LLM up-scaling methods that use mixture-of-experts, DUS does not require complex changes to train and inference efficiently. We show experimentally that DUS is simple yet effective in scaling up high-performance LLMs from small ones. Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, a variant fine-tuned for instruction-following capabilities, surpassing Mixtral-8x7B-Instruct. SOLAR 10.7B is publicly available under the Apache 2.0 license, promoting broad access and application in the LLM field.
~text~"""
    }
]

responses = []
for i in range(2):
    response = get_completion(messages=message)
    responses.append(response)

- When using unusual format symbols or marks, the stability of the results decreases each time they are generated.

In [38]:
for i in range(2):
    print(f"n = {i+1}")
    print(responses[i])
    print("\n")

n = 1
company names: SOLAR

personal names: None

specific topics: large language model (LLM), natural language processing (NLP), depth up-scaling (DUS), mixture-of-experts, instruction-following

themes: scaling up LLMs, DUS method, SOLAR 10.7B-Instruct, Apache 2.0 license


n = 2
company names:
* SOLAR

personal names:
* None

specific topics:
* large language model (LLM)
* natural language processing (NLP)
* depth up-scaling (DUS)
* mixture-of-experts
* instruction-following

themes:
* scaling up LLMs
* demonstrating superior performance in various NLP tasks
* efficient up-scaling of LLMs
* simple yet effective methods for scaling up high-performance LLMs
* fine-tuning for instruction-following capabilities
* promoting broad access and application in the LLM field




**References**

Brown, T., Mann, B., Ryder, N., Subbiah, M., Kaplan, J. D., Dhariwal, P., ... & Amodei, D. (2020). *Language models are few-shot learners.* Advances in Neural Information Processing Systems, 33, 1877-1901.  
Wei, J., Bosma, M., Zhao, V. Y., Guu, K., Yu, A. W., Lester, B., ... & Le, Q. V. (2021). *Finetuned language models are zero-shot learners.* arXiv preprint arXiv:2109.01652.  
Work, W. M. I. C. L. *Rethinking the Role of Demonstrations: What Makes In-Context Learning Work?*